In [5]:
import pandas as pd 
import numpy as np  
import re
import ssl
import pycountry
import pycountry_convert as pc
from toolkit import * 
from shapely.geometry import Point
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx

In [3]:
df=load_data()

In [4]:
df.isna().sum() 

transaction_id                        0
timestamp                             0
user_id                               0
merchant_id                           0
amount                                0
channel                               0
currency                              0
device                                0
location                              0
payment_method                        0
is_international                      0
session_length_seconds                0
is_first_time_merchant                0
is_fraud                              0
age                                   0
sex                                   0
education                         96566
primary_source_of_income              0
sum_of_monthly_installments           0
sum_of_monthly_expenses               0
country_x                             0
signup_date                           0
risk_score                            0
category                              0
country_y                             0


In [3]:
df["education"]=df["education"].fillna("basic education")

In [4]:
df.columns 

Index(['transaction_id', 'timestamp', 'user_id', 'merchant_id', 'amount',
       'channel', 'currency', 'device', 'location', 'payment_method',
       'is_international', 'session_length_seconds', 'is_first_time_merchant',
       'is_fraud', 'age', 'sex', 'education', 'primary_source_of_income',
       'sum_of_monthly_installments', 'sum_of_monthly_expenses', 'country_x',
       'signup_date', 'risk_score', 'category', 'country_y', 'trust_score',
       'number_of_alerts_last_6_months', 'avg_transaction_amount',
       'account_age_months', 'has_fraud_history'],
      dtype='object')

### Adding countries to dataframe

In [5]:
df.rename(  columns={'country_x': 'country_user','country_y':'country_merchant'},
    inplace=True)

In [7]:
df['location'] 

0         {'lat': 53.582891, 'long': 3.753494}
1         {'lat': 52.926229, 'long': 8.058399}
2         {'lat': 35.095503, 'long': 16.19487}
3         {'lat': 36.49444, 'long': -8.382611}
4        {'lat': 62.680745, 'long': -0.389401}
                         ...                  
49995    {'lat': 46.316348, 'long': 32.578442}
49996    {'lat': 50.093219, 'long': 12.730941}
49997    {'lat': 39.605503, 'long': -0.406485}
49998     {'lat': 55.301073, 'long': 2.378804}
49999    {'lat': 56.042572, 'long': 12.953562}
Name: location, Length: 500000, dtype: object

In [8]:

df['lat'] = df['location'].apply(
    lambda s: float(re.search(
        r"'lat'\s*:\s*([-+]?\d*\.\d+|\d+)",
        str(s)
    ).group(1))
)

df['lon'] = df['location'].apply(
    lambda s: float(re.search(
        r"'long'\s*:\s*([-+]?\d*\.\d+|\d+)",
        str(s)
    ).group(1))
)


In [9]:
ssl._create_default_https_context = ssl._create_unverified_context
countries_url = (
    "https://raw.githubusercontent.com/datasets/geo-countries/master/data/countries.geojson"
)
world = gpd.read_file(countries_url)
ocean_url = "https://naturalearth.s3.amazonaws.com/10m_physical/ne_10m_ocean.zip"
ocean = gpd.read_file(ocean_url)

In [10]:
world = world[['name', 'ISO3166-1-Alpha-3', 'geometry']]
world = world.rename(columns={
    'name': 'ADMIN',
    'ISO3166-1-Alpha-3': 'ISO_A3'
})

In [11]:

gdf = gpd.GeoDataFrame(
    df,
    geometry=[Point(lon, lat) for lon, lat in zip(df['lon'], df['lat'])],
    crs="EPSG:4326"
)

In [12]:
gdf

,transaction_id,timestamp,user_id,merchant_id,amount,channel,currency,device,location,payment_method,...,category,country_merchant,trust_score,number_of_alerts_last_6_months,avg_transaction_amount,account_age_months,has_fraud_history,lat,lon,geometry
0,TX350000,2023-02-09 07:19:00,U00254,M0727,57.54,online,EUR,iOS,"{'lat': 53.582891, 'long': 3.753494}",bank_transfer,...,electronics,Finland,0.770458,1,24.72,119,1,53.582891,3.753494,POINT (3.75349 53.58289)
1,TX350001,2022-02-19 21:09:00,U07272,M0102,42.60,in-store,EUR,iOS,"{'lat': 52.926229, 'long': 8.058399}",credit_card,...,restaurants,Italy,0.416960,2,18.74,92,1,52.926229,8.058399,POINT (8.0584 52.92623)
2,TX350002,2023-03-24 04:44:00,U04391,M0081,32.51,online,EUR,Web,"{'lat': 35.095503, 'long': 16.19487}",bank_transfer,...,grocery,Sweden,0.450369,5,89.82,60,1,35.095503,16.194870,POINT (16.19487 35.0955)
3,TX350003,2023-05-27 18:50:00,U19889,M0698,45.49,in-store,EUR,Android,"{'lat': 36.49444, 'long': -8.382611}",bank_transfer,...,education,Spain,0.701037,5,54.97,115,0,36.494440,-8.382611,POINT (-8.38261 36.49444)
4,TX350004,2023-10-13 12:30:00,U07427,M0231,56.58,mobile,EUR,Web,"{'lat': 62.680745, 'long': -0.389401}",debit_card,...,grocery,Portugal,0.505832,6,113.27,33,0,62.680745,-0.389401,POINT (-0.3894 62.68074)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,TX299995,2023-08-18 04:05:00,U15812,M0082,17.83,mobile,EUR,Web,"{'lat': 46.316348, 'long': 32.578442}",bank_transfer,...,education,Netherlands,0.882334,7,142.95,24,0,46.316348,32.578442,POINT (32.57844 46.31635)
49996,TX299996,2023-08-18 20:04:00,U07618,M0055,76.66,in-store,EUR,Web,"{'lat': 50.093219, 'long': 12.730941}",mobile_payment,...,clothing,Italy,0.566883,2,65.19,98,1,50.093219,12.730941,POINT (12.73094 50.09322)
49997,TX299997,2023-09-13 23:42:00,U15747,M0860,14.53,mobile,EUR,Web,"{'lat': 39.605503, 'long': -0.406485}",bank_transfer,...,clothing,Portugal,0.629327,1,63.76,50,1,39.605503,-0.406485,POINT (-0.40648 39.6055)
49998,TX299998,2023-04-15 13:27:00,U11034,M0257,9.26,in-store,EUR,Web,"{'lat': 55.301073, 'long': 2.378804}",debit_card,...,gaming,Finland,0.617473,4,98.30,29,1,55.301073,2.378804,POINT (2.3788 55.30107)


In [13]:
pts = gpd.sjoin(
    gdf,
    world,
    how='left',
    predicate='within'
)

In [14]:
pts['ADMIN'] = pts['ADMIN'].fillna('Ocean')

In [15]:
pts 

,transaction_id,timestamp,user_id,merchant_id,amount,channel,currency,device,location,payment_method,...,number_of_alerts_last_6_months,avg_transaction_amount,account_age_months,has_fraud_history,lat,lon,geometry,index_right,ADMIN,ISO_A3
0,TX350000,2023-02-09 07:19:00,U00254,M0727,57.54,online,EUR,iOS,"{'lat': 53.582891, 'long': 3.753494}",bank_transfer,...,1,24.72,119,1,53.582891,3.753494,POINT (3.75349 53.58289),NaN,Ocean,NaN
1,TX350001,2022-02-19 21:09:00,U07272,M0102,42.60,in-store,EUR,iOS,"{'lat': 52.926229, 'long': 8.058399}",credit_card,...,2,18.74,92,1,52.926229,8.058399,POINT (8.0584 52.92623),49.0,Germany,DEU
2,TX350002,2023-03-24 04:44:00,U04391,M0081,32.51,online,EUR,Web,"{'lat': 35.095503, 'long': 16.19487}",bank_transfer,...,5,89.82,60,1,35.095503,16.194870,POINT (16.19487 35.0955),NaN,Ocean,NaN
3,TX350003,2023-05-27 18:50:00,U19889,M0698,45.49,in-store,EUR,Android,"{'lat': 36.49444, 'long': -8.382611}",bank_transfer,...,5,54.97,115,0,36.494440,-8.382611,POINT (-8.38261 36.49444),NaN,Ocean,NaN
4,TX350004,2023-10-13 12:30:00,U07427,M0231,56.58,mobile,EUR,Web,"{'lat': 62.680745, 'long': -0.389401}",debit_card,...,6,113.27,33,0,62.680745,-0.389401,POINT (-0.3894 62.68074),NaN,Ocean,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,TX299995,2023-08-18 04:05:00,U15812,M0082,17.83,mobile,EUR,Web,"{'lat': 46.316348, 'long': 32.578442}",bank_transfer,...,7,142.95,24,0,46.316348,32.578442,POINT (32.57844 46.31635),33.0,Ukraine,UKR
49996,TX299996,2023-08-18 20:04:00,U07618,M0055,76.66,in-store,EUR,Web,"{'lat': 50.093219, 'long': 12.730941}",mobile_payment,...,2,65.19,98,1,50.093219,12.730941,POINT (12.73094 50.09322),48.0,Czechia,CZE
49997,TX299997,2023-09-13 23:42:00,U15747,M0860,14.53,mobile,EUR,Web,"{'lat': 39.605503, 'long': -0.406485}",bank_transfer,...,1,63.76,50,1,39.605503,-0.406485,POINT (-0.40648 39.6055),66.0,Spain,ESP
49998,TX299998,2023-04-15 13:27:00,U11034,M0257,9.26,in-store,EUR,Web,"{'lat': 55.301073, 'long': 2.378804}",debit_card,...,4,98.30,29,1,55.301073,2.378804,POINT (2.3788 55.30107),NaN,Ocean,NaN


In [17]:
def country_to_continent(country_name):
    if not country_name or country_name == "Ocean":
        return "Ocean"
    try:
        c = pycountry.countries.lookup(country_name)
        alpha2 = c.alpha_2
        cont_code = pc.country_alpha2_to_continent_code(alpha2)
        return pc.convert_continent_code_to_continent_name(cont_code)
    except (LookupError, KeyError):
        return None

In [18]:
pts['continent_transaction'] = pts['ADMIN'].apply(country_to_continent)
pts['continent_user']=pts['country_user'].apply(country_to_continent) 
pts['continent_merchant']=pts['country_merchant'].apply(country_to_continent)

In [19]:
pts=pts.rename(columns={'ADMIN':'country_transaction'})

In [20]:
pts.columns 

Index(['transaction_id', 'timestamp', 'user_id', 'merchant_id', 'amount',
       'channel', 'currency', 'device', 'location', 'payment_method',
       'is_international', 'session_length_seconds', 'is_first_time_merchant',
       'is_fraud', 'age', 'sex', 'education', 'primary_source_of_income',
       'sum_of_monthly_installments', 'sum_of_monthly_expenses',
       'country_user', 'signup_date', 'risk_score', 'category',
       'country_merchant', 'trust_score', 'number_of_alerts_last_6_months',
       'avg_transaction_amount', 'account_age_months', 'has_fraud_history',
       'lat', 'lon', 'geometry', 'index_right', 'country_transaction',
       'ISO_A3', 'continent_transaction', 'continent_user',
       'continent_merchant'],
      dtype='object')

In [21]:
pts.head()

,transaction_id,timestamp,user_id,merchant_id,amount,channel,currency,device,location,payment_method,...,has_fraud_history,lat,lon,geometry,index_right,country_transaction,ISO_A3,continent_transaction,continent_user,continent_merchant
0,TX350000,2023-02-09 07:19:00,U00254,M0727,57.54,online,EUR,iOS,"{'lat': 53.582891, 'long': 3.753494}",bank_transfer,...,1,53.582891,3.753494,POINT (3.75349 53.58289),NaN,Ocean,NaN,Ocean,Europe,Europe
1,TX350001,2022-02-19 21:09:00,U07272,M0102,42.60,in-store,EUR,iOS,"{'lat': 52.926229, 'long': 8.058399}",credit_card,...,1,52.926229,8.058399,POINT (8.0584 52.92623),49.0,Germany,DEU,Europe,Europe,Europe
2,TX350002,2023-03-24 04:44:00,U04391,M0081,32.51,online,EUR,Web,"{'lat': 35.095503, 'long': 16.19487}",bank_transfer,...,1,35.095503,16.194870,POINT (16.19487 35.0955),NaN,Ocean,NaN,Ocean,Europe,Europe
3,TX350003,2023-05-27 18:50:00,U19889,M0698,45.49,in-store,EUR,Android,"{'lat': 36.49444, 'long': -8.382611}",bank_transfer,...,0,36.494440,-8.382611,POINT (-8.38261 36.49444),NaN,Ocean,NaN,Ocean,Europe,Europe
4,TX350004,2023-10-13 12:30:00,U07427,M0231,56.58,mobile,EUR,Web,"{'lat': 62.680745, 'long': -0.389401}",debit_card,...,0,62.680745,-0.389401,POINT (-0.3894 62.68074),NaN,Ocean,NaN,Ocean,Europe,Europe


In [22]:
pts.shape

(500000, 39)

In [23]:
pts.drop(columns=['index_right','ISO_A3','location','lon','lat','location'],inplace=True,axis=1)

In [24]:
pts.columns 

Index(['transaction_id', 'timestamp', 'user_id', 'merchant_id', 'amount',
       'channel', 'currency', 'device', 'payment_method', 'is_international',
       'session_length_seconds', 'is_first_time_merchant', 'is_fraud', 'age',
       'sex', 'education', 'primary_source_of_income',
       'sum_of_monthly_installments', 'sum_of_monthly_expenses',
       'country_user', 'signup_date', 'risk_score', 'category',
       'country_merchant', 'trust_score', 'number_of_alerts_last_6_months',
       'avg_transaction_amount', 'account_age_months', 'has_fraud_history',
       'geometry', 'country_transaction', 'continent_transaction',
       'continent_user', 'continent_merchant'],
      dtype='object')

In [25]:
pts['continent_transaction']=pts['continent_transaction'].fillna('Asia') 

In [26]:
pts.sort_values("timestamp", ascending=True, inplace=True)

In [27]:
pts.head()

,transaction_id,timestamp,user_id,merchant_id,amount,channel,currency,device,payment_method,is_international,...,trust_score,number_of_alerts_last_6_months,avg_transaction_amount,account_age_months,has_fraud_history,geometry,country_transaction,continent_transaction,continent_user,continent_merchant
47783,TX497783,2022-01-01 00:06:00,U04367,M0197,41.78,in-store,EUR,Android,mobile_payment,1,...,0.867635,1,85.26,79,1,POINT (36.09244 53.68845),Russia,Asia,Europe,Europe
37684,TX087684,2022-01-01 00:12:00,U07964,M0056,77.36,online,EUR,Android,debit_card,1,...,0.756445,3,24.93,69,0,POINT (13.10645 68.95111),Ocean,Ocean,Europe,Europe
16651,TX466651,2022-01-01 00:12:00,U02226,M0496,160.69,in-store,EUR,Android,bank_transfer,1,...,0.887738,2,94.39,79,1,POINT (-1.28857 35.87533),Ocean,Ocean,Europe,Europe
5076,TX005076,2022-01-01 00:13:00,U04685,M0484,45.19,in-store,EUR,Web,debit_card,1,...,0.596116,5,23.05,31,1,POINT (38.06087 60.43793),Russia,Asia,Europe,Europe
29480,TX429480,2022-01-01 00:14:00,U13341,M0960,10.76,online,EUR,Web,bank_transfer,1,...,0.857549,0,82.35,82,1,POINT (14.58235 40.25572),Ocean,Ocean,Europe,Europe


Saving data

In [29]:
for i in range(10):
    df_ = pts.iloc[i*50_000:(i+1)*50_000, :]
    df_.to_csv(f"Cleaned_data/clean_{i+1:02}.csv", index=False)